In [5]:
import numpy as np

In [22]:
%run ./observing_stars.ipynb

INFO: Query finished. [astroquery.utils.tap.core]
1     0.732477
4     0.664991
7     0.597505
10    0.530018
13    0.462532
dtype: float64


Based on Domínguez-Palmero, L. et al. (2018) Design of the observation queue scheduler for WEAVE on the WHT, p. 34. https://doi.org/10.1117/12.2312538.

Score of a specific star :

$$S_{total} = \sum_{i=1}^{n} \omega_{i} f_{i}(t) \quad \text{if all} \quad f_{i}(t)\neq0$$

$$S_{total} = 0 \quad \text{if any} \quad f_{i}(t)=0$$

| Parameter        | Scoring function | Contraints in the OB |
|------------------|------------------|---------------------------------------------------------------|
| Seeing           | $f_{seeing}$     | Max seeing: $seeing_{OBmax}$                                  |
| Sky Brightness   | $f_{sb}$         | Brightest sky: $sb_{OBbright}$                                |
| Sky Transparency | $f_{stra}$       | Minimum sky transparency: $stra_{OBmin}$                      |
| Hour Angle range | $f_{HA}$         | Earliest and latest hour angles: $HA_{OBmin}, HA_{OBmax}$     |
| Priority         | $f_{OBtp}$       | Science Priority: $P_{OBsci}$ Override Priority: $P_{OBtovr}$ |
| Overheads        | $f_{overheads}$  | -                                                             |

In [ ]:
# HARD CONSTRAINTS:

# Telescope constraints
# Instrument constraints
# Minimum elevation
# Minimum angular distance of the moon
# Observing-conditions limits

# ALSO NEED TO ADD:

# Phase difference from next closest observation
# Gradient of modelled flux

In [16]:
# Scoring function for seeing:

def seeing_score(airmass, seeing_zenith, maximum_seeing, seeing_max, seeing_min):
    seeing = airmass**0.6 * seeing_zenith
    if seeing > maximum_seeing:
        return 0
    if seeing <= maximum_seeing:
        seeing_score  = (seeing_max - maximum_seeing + seeing_min) / ((maximum_seeing - seeing_min + 1) * (seeing_max - maximum_seeing + seeing))
        return seeing_score
    return 0

In [34]:
# Scoring function for sky brightness:

def sky_brightness_score(sky_brightness, brightest_sky, sb_min):
    if sky_brightness >= brightest_sky:
        sky_brightness_score = (brightest_sky + 1) / ((sb_min - brightest_sky + 1) * (sb_min - sky_brightness + brightest_sky + 1))
        return sky_brightness_score
    return 0

In [4]:
# Scoring function for sky transparency:

def sky_trans_score(sky_trans, min_sky_trans, sky_trans_max):
    if sky_trans >= min_sky_trans:
        sky_trans_score = (min_sky_trans + 1) / ((sky_trans_max - min_sky_trans + 1) * (sky_trans_max - sky_trans + min_sky_trans + 1))
        return sky_trans_score
    if sky_trans < min_sky_trans:
        return 0
    return 0

In [5]:
# Scoring function for hour angle:

def hour_angle_score(HA_t, HA_max, HA_min):
    if HA_t < HA_min and HA_t > HA_max:
        return 0
    HA_H = 0
    if HA_min <= 0 and HA_max >= 0:
        HA_H = 0
    elif HA_min < 0 and HA_max < 0:
        HA_H = HA_max
    elif HA_min > 0 and HA_max > 0:
        HA_H = HA_min
    HA_range = np.absolute(HA_max - HA_min)
    HA_dist = np.absolute(HA_t - HA_H)
    hour_angle_score = 1 / ((HA_range + 1) * (HA_dist + 1))
    return hour_angle_score

In [6]:
# Scoring function for priority:

def priority_score(science_priority, override_priority):
    priority_score = science_priority * override_priority
    return priority_score

In [7]:
# Scoring function for overheads:

def overheads_score(overheads):
    overheads_score = 1 - (0.9 * overheads)
    return overheads_score

In [18]:
# Scoring function for phase offset:

def phase_offset_score(phase_offset):
    phase_offset_score = np.round(np.square(np.sin(np.pi * phase_offset)), 10)
    return phase_offset_score

In [67]:
# Total score function:

def total_score(scoring_functions):
    total_score = 0
    for function, weight in scoring_functions:
        if function == 0:
            total_score = 0
            return total_score
        total_score += weight * function
    return total_score

In [19]:
phase_score = phase_offset_score(0.333)
phase_score

np.float64(0.7490925527)

In [141]:
seeing_score_star = seeing_score(1, 1, 1, 3, 0.3)
sky_brightness_score_star = sky_brightness_score(20.5, 20.5, 22)
sky_trans_score_star = sky_trans_score(0.7, 0.7, 0.88)
hour_angle_score_star = hour_angle_score(0, 1.5, -0.5)
priority_score_star = priority_score(1, 1)
overheads_score_star = overheads_score(0)

functions_star = np.array([
    [seeing_score_star, 1],
    [sky_brightness_score_star, 1],
    [sky_trans_score_star, 1],
    [hour_angle_score_star, 1],
    [priority_score_star, 1],
    [overheads_score_star, 1]
])

0.3739130434782609

In [104]:
final_score = total_score(functions_star)
final_score

np.float64(3.924544836043826)

In [23]:
phase_offset = find_phase_offset(RZ_Cep_data, current_time)
phase_offset_score_star = phase_offset_score(phase_offset)
phase_offset_score_star

1     0.732477
4     0.664991
7     0.597505
10    0.530018
13    0.462532
dtype: float64


np.float64(0.5549380675)